<a href="https://colab.research.google.com/github/educardenas97/API/blob/experimental/Model_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.mspbs.gov.py/xweb/vacunados.csv  -P /tmp/ -o vacunados.csv --no-check-certificate

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import os

csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vacunados/dataset/vacunados_21_nov.csv', sep=';')
csv_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vacunados/dataset/padron.csv', sep=',')

In [88]:
#Filter data
data = csv.copy()
data = data.drop_duplicates()
data = data[data['cedula'].astype(str).str.isdigit()]
data['cedula'] = pd.to_numeric(data['cedula'])
data['dosis'] = data['dosis'].astype(int)
data['fecha_aplicacion'] = pd.to_datetime(data['fecha_aplicacion']).dt.date

data['semana'] = data['fecha_aplicacion'] -  data['fecha_aplicacion'].min()
data['semana'] = (data['semana']).dt.days
data = data[data['cedula']<90000000]
data = data[data['dosis'] == 1]


padron = csv_2.copy().drop(columns='date')
join_dataframe = data.join(padron.set_index('id'), on='cedula', how='inner')
join_dataframe.drop(inplace=True, columns=['actualizado_al','fecha_aplicacion','nombre','apellido','establecimiento'])

print(join_dataframe.columns)

Index(['cedula', 'dosis', 'descripcion_vacuna', 'semana', 'year'], dtype='object')


In [ ]:
#Distribución
#join_dataframe['year'].plot.kde()
#Primera parte
mean = join_dataframe[join_dataframe['dosis'] == 1]
view = mean.groupby('descripcion_vacuna').median()
print(view.sort_values(by='year'))
#Edad vs vacunas
fig, ax = plt.subplots()
ax.scatter(view['year'], view.index)
ax.set_xlabel('edad')
#Semanas vs edad
fig, ax = plt.subplots()
view = mean.groupby('semana').median()
ax.scatter(view.index, view['year'])
ax.set_ylabel('edad')
ax.set_xlabel('semana')
#Semanas vs cedula
fig, ax = plt.subplots()
view = mean.groupby('semana').median()
ax.scatter(view.index, view['cedula'], c='green')
ax.set_ylabel('cedula')
ax.set_xlabel('semana')


In [82]:
import plotly.express as px
fig = px.scatter_3d(join_dataframe.sample(n=199), x='cedula', y='semana', z='year', color='descripcion_vacuna')
fig.show()

In [95]:
!pip install category_encoders
import category_encoders as ce

encoder=ce.OneHotEncoder(cols='descripcion_vacuna',handle_unknown='return_nan',return_df=True,use_cat_names=True)

#encoder=ce.sum_coding.SumEncoder(cols='City',verbose=False,)
encoder.fit_transform(join_dataframe)

,cedula,dosis,descripcion_vacuna_ASTRAZENECA-CHADOX1-S - COVID-19,descripcion_vacuna_PFIZER - COVID -19,descripcion_vacuna_SPUTNIK V COVID-19,descripcion_vacuna_COVAXIN COVID-19,descripcion_vacuna_MODERNA - COVID -19,descripcion_vacuna_CORONAVAC COVID-19,descripcion_vacuna_HAYAT VAX COVID-19,descripcion_vacuna_SINOPHARM COVID-19,semana,year
51,7310622,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263,2002
65,1950766,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263,1972
66,2205376,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263,1986
72,2313648,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263,1985
100,2004560,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263,1975
...,...,...,...,...,...,...,...,...,...,...,...,...
5643371,2640774,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263,1964
5643399,5047930,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,255,1993
5643591,6824161,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263,2003
5643598,4517259,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263,1981
